In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F

In [2]:
wine_data = pd.read_csv('data/wine_data.csv')

In [3]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [4]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [5]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()

In [6]:
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

## Creating a classifier


In [7]:
input_size = 13
output_size = 3
hidden_size = 100

In [8]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

In [9]:
model = Net()

In [10]:
optimizer = optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.NLLLoss()

In [11]:
checkpoint = torch.load('wine_checkpoint.pth')

In [12]:
model.load_state_dict(checkpoint['model_state_dict'])

In [13]:
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [14]:
start_epoch = checkpoint['epoch']
start_epoch

99

In [15]:
loss = checkpoint['loss']
loss

tensor(0.2982, requires_grad=True)

In [16]:
model.train()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [17]:
epochs = 500

for epoch in range(start_epoch, epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    if epoch % 100 == 0:
        print ('Epoch', epoch, 'loss', loss.item())

Epoch 100 loss 0.2845788300037384
Epoch 200 loss 0.03398529440164566
Epoch 300 loss 0.11083189398050308
Epoch 400 loss 0.07427312433719635


In [18]:
predict_out = model(Xtest_)
_, predict_y = torch.max(predict_out, 1)

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print ('prediction accuracy', accuracy_score(Ytest_.data, predict_y.data))
print ('micro precision', precision_score(Ytest_.data, predict_y.data, average='micro'))
print ('micro recall', recall_score(Ytest_.data, predict_y.data, average='micro'))

prediction accuracy 0.9027777777777778
micro precision 0.9027777777777778
micro recall 0.9027777777777778
